In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

os.environ["TOKENIZERS_PARALLELISM"] = "false"
db_embedding = HuggingFaceEmbeddings(model_name='../models/text2vec-large-chinese/')
db_directory = '../vectordb/db_DBQA'

No sentence-transformers model found with name ../models/text2vec-large-chinese/. Creating a new one with MEAN pooling.


## 向量数据库建立

In [13]:
!rm -r ../vectordb/db_DBQA

In [14]:
from PDFLoader import PDFLoader

file_path = '../files/'
file_list = ['2023中国市场招聘趋势.pdf',
             '数据要素白皮书-2022.pdf',
             '人工智能生成内容白皮书-2022.pdf']

vectordb = Chroma(embedding_function=db_embedding, persist_directory=db_directory)

docs = []
for file in file_list:
    pdf_path = file_path + file
    docs = PDFLoader(pdf_path, max_len=200)
    
    vectordb.add_documents(docs)
    vectordb.persist()

# vectordb = Chroma.from_documents(documents=docs, embedding=db_embedding, persist_directory=db_directory)
# vectordb.persist()

Using embedded DuckDB with persistence: data will be stored in: ../vectordb/db_DBQA


## 向量数据库查询

In [88]:
vectordb = None
vectordb = Chroma(embedding_function=db_embedding, persist_directory=db_directory)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

Using embedded DuckDB with persistence: data will be stored in: ../vectordb/db_DBQA


In [89]:
query = '介绍一下虚拟歌手洛天依'
docs = retriever.get_relevant_documents(query)

In [90]:
docs

[Document(page_content='VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，都是能够让粉丝深度参与共创的虚拟歌手以洛天依为例，任何人通过声库创作词曲，都能达到“洛天依演唱一首歌”的效果从2012年7月12日洛天依出道至今十年的时间内，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系二是通过', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
 Document(page_content='101同框合影的活动，这些互动的内容极大地激发出了用户的情感，带来了社交传播的迅速破圈在语音合成方面，变声增加互动娱乐性如QQ等多款社交软件、和平精英等多款游戏均已集成变声功能，支持用户体验大叔、萝莉等多种不同声线，让沟通成为一种乐此不疲的游戏打造虚拟偶像，释放IP价值一是实现与用户共创合成歌曲，人工智能生成内容（AIGC）白皮书36不断加深粉丝黏性以初音未来和洛天依为代表的“虚拟歌姬”，都是基于', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
 Document(page_content='如欧莱雅、飞利浦、完美日记等品牌的虚拟主播一般会在凌晨0点上线，并进行近9个小时的直播，与真人主播形成了24小时无缝对接的直播服务二是虚拟化的品牌主播更能加速店铺或品牌年轻化进程，拉近与新消费人群的距离，塑造元宇宙时代的店铺形象，未来可通过延展应用到元宇宙中更多元的虚拟场景，实现多圈层传播如彩妆品牌“卡姿兰”推出自己的品牌虚拟形象，并将其引入直播间作为其天猫旗舰店日常的虚拟主播导购', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'})]

## 模型加载

In [59]:
from ChatGLM import ChatGLM

In [4]:
llm = ChatGLM(model_path='../models/chatglm-6b-int4/')

No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


## Prompt 整合

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [16]:
prompt_template="""请评估提问与上下文信息的相关性，根据上下文内容对提问进行回答，如果提问与提供的上下文无关，请回答"我不知道"。在回答过程中，保持准确性和专业性，避免回答无关答案和冗长的答案：
    上下文: {context}
    提问: {query}
    回答:"""

prompt = PromptTemplate(
    input_variables=["context", "query"],
    template=prompt_template
)

chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

In [17]:
query = '请简单地介绍一下虚拟歌手洛天依'
docs = retriever.get_relevant_documents(query)

In [18]:
result = chain({"input_documents": docs, "query": query}, return_only_outputs=True)

In [19]:
print(result['output_text'])

洛天依是一位由 VOCALOID 语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。洛天依出道于 2012 年 7 月 12 日，至今已经十年的时间。音乐人和粉丝为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。此外，洛天依还是一位虚拟歌手，通过与用户共创合成歌曲，人工智能生成内容(AIGC)白皮书等方式，不断加深粉丝黏性。以洛天依为代表的“虚拟歌姬”，是 基于 VOCALOID 语音合成引擎软件的虚拟人物，能够让粉丝深度参与共创。


## ConversationalRetrievalChain

In [107]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [108]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory)

In [109]:
query = '请简单地介绍一下虚拟歌手洛天依'
result = chain({"question": query})

In [110]:
result

{'question': '请简单地介绍一下虚拟歌手洛天依',
 'chat_history': [HumanMessage(content='请简单地介绍一下虚拟歌手洛天依', additional_kwargs={}, example=False),
  AIMessage(content='洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。', additional_kwargs={}, example=False)],
 'answer': '洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。'}

In [111]:
query = '她是什么时候出道的？'
result = chain({"question": query})

In [112]:
result

{'question': '她是什么时候出道的？',
 'chat_history': [HumanMessage(content='请简单地介绍一下虚拟歌手洛天依', additional_kwargs={}, example=False),
  AIMessage(content='洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。', additional_kwargs={}, example=False),
  HumanMessage(content='她是什么时候出道的？', additional_kwargs={}, example=False),
  AIMessage(content='In 2012', additional_kwargs={}, example=False)],
 'answer': 'In 2012'}

In [113]:
result['answer']

'In 2012'

## With Sources

In [91]:
memory = ConversationBufferMemory(output_key='answer', memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever, memory=memory, return_source_documents=True)

In [92]:
query = '请简单地介绍一下虚拟歌手洛天依'
result = qa({"question": query})

In [93]:
result

{'question': '请简单地介绍一下虚拟歌手洛天依',
 'chat_history': [HumanMessage(content='请简单地介绍一下虚拟歌手洛天依', additional_kwargs={}, example=False),
  AIMessage(content='洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。', additional_kwargs={}, example=False)],
 'answer': '洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。',
 'source_documents': [Document(page_content='VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，都是能够让粉丝深度参与共创的虚拟歌手以洛天依为例，任何人通过声库创作词曲，都能达到“洛天依演唱一首歌”的效果从2012年7月12日洛天依出道至今十年的时间内，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系二是通过', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
  Document(page_content='101同框合影的活动，这些互动的内容极大地激发出了用户的情感，带来了社交传播的迅速破圈在语音合成方面，变声增加互动娱乐性如QQ等多款社交软件、和平精英等多款游戏均已集成变声功能，支持用户体验大叔、萝莉等多种不同声线，让沟通成为一种乐此不疲的游戏打造虚拟偶像，释放IP价值一是实现与用

In [94]:
query = '2022年国内新能源车的渗透率是多少'
result = qa({"question": query})

In [95]:
result

{'question': '2022年国内新能源车的渗透率是多少',
 'chat_history': [HumanMessage(content='请简单地介绍一下虚拟歌手洛天依', additional_kwargs={}, example=False),
  AIMessage(content='洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。', additional_kwargs={}, example=False),
  HumanMessage(content='2022年国内新能源车的渗透率是多少', additional_kwargs={}, example=False),
  AIMessage(content="I don't have information on the specific percentage of total vehicle sales that were electric vehicles in 2022. The Chinese government has set ambitious goals for the development of electric vehicles and the country has made significant progress in this area. In 2021, the Chinese government launched a new program to increase the number of electric vehicles on the road by 2025, with a goal to have 50% of all vehicle sales being electric by then. Additionally, the country has also made it easier to purchase and own electric vehi

In [96]:
result['answer']

"I don't have information on the specific percentage of total vehicle sales that were electric vehicles in 2022. The Chinese government has set ambitious goals for the development of electric vehicles and the country has made significant progress in this area. In 2021, the Chinese government launched a new program to increase the number of electric vehicles on the road by 2025, with a goal to have 50% of all vehicle sales being electric by then. Additionally, the country has also made it easier to purchase and own electric vehicles, with the introduction of new regulations and policies to support the development of the electric vehicle industry."

In [97]:
query = '请换成中文来回答'
result = qa({"question": query})

In [98]:
result

{'question': '请换成中文来回答',
 'chat_history': [HumanMessage(content='请简单地介绍一下虚拟歌手洛天依', additional_kwargs={}, example=False),
  AIMessage(content='洛天依是一个由VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，能够让粉丝深度参与共创。从2012年7月12日洛天依出道至今，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系。洛天依的出道引起了全球范围内的关注，并成为了许多粉丝的虚拟偶像。', additional_kwargs={}, example=False),
  HumanMessage(content='2022年国内新能源车的渗透率是多少', additional_kwargs={}, example=False),
  AIMessage(content="I don't have information on the specific percentage of total vehicle sales that were electric vehicles in 2022. The Chinese government has set ambitious goals for the development of electric vehicles and the country has made significant progress in this area. In 2021, the Chinese government launched a new program to increase the number of electric vehicles on the road by 2025, with a goal to have 50% of all vehicle sales being electric by then. Additionally, the country has also made it easier to purchase and own electric vehicles, with

In [99]:
result['answer']

'2022年国内新能源车的渗透率是25%.'

In [22]:
vectordb = None
vectordb = Chroma(embedding_function=db_embedding, persist_directory=db_directory)
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

Using embedded DuckDB with persistence: data will be stored in: ../vectordb/db_DBQA


In [23]:
query = '2022新能源汽车渗透率是多少'
docs = retriever.get_relevant_documents(query)

In [24]:
docs

[Document(page_content='2022年注定是新能源车及其产业链提前进入激烈竞争的一年，既是技术产品营销的竞争，亦是人才争夺的竞争这一年，国内新能源车的渗透率突破25%，提前达到三年后目标KOS新能源招聘团队认为，在2022人才争夺战中，新能源车人才竞争的42三个战场会分别是研发、交付、营销各大车企招聘的增量直接反映其业务发展势头，直观例子是BYD在2022年的销量全面赶超特斯拉，而BYD在招聘市场上的热度亦然如此', metadata={'source': '../files/2023中国市场招聘趋势.pdf'}),
 Document(page_content='关于加强数字政府建设的指导意见[EB/OL].，2022.http://www.gov.cn/zhengce/content/2022-06/23/content_5697299.htm.[5]中国信息通信研究院.大数据白皮书[R].2021.[6]隐私计算联盟，中国信通院云大所.隐私计算白皮书（2021年）[R].2021.[7]华中师范大学信息管理学院，湖北省数据治理与智能决策研究中心.', metadata={'source': '../files/数据要素白皮书-2022.pdf'}),
 Document(page_content='.“十四五”数字经济发展规划[EB/OL].2021.http://www.gov.cn/zhengce/content/2022-01/12/content_5667817.htm.[3]中共中央，国务院.关于加快建设全国统一大市场的意见[EB/OL].2022.http://www.gov.cn/gongbao/content/2022/content_5687499.htm.[4]国务院.', metadata={'source': '../files/数据要素白皮书-2022.pdf'})]

In [25]:
query = '介绍一下虚拟歌手洛天依'
docs = retriever.get_relevant_documents(query)

In [26]:
docs

[Document(page_content='VOCALOID语音合成引擎软件为基础创造出来的虚拟人物，由真人提供声源，再由软件合成人声，都是能够让粉丝深度参与共创的虚拟歌手以洛天依为例，任何人通过声库创作词曲，都能达到“洛天依演唱一首歌”的效果从2012年7月12日洛天依出道至今十年的时间内，音乐人以及粉丝已为洛天依创作了超过一万首作品，通过为用户提供更多想象和创作空间的同时，与粉丝建立了更深刻联系二是通过', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
 Document(page_content='101同框合影的活动，这些互动的内容极大地激发出了用户的情感，带来了社交传播的迅速破圈在语音合成方面，变声增加互动娱乐性如QQ等多款社交软件、和平精英等多款游戏均已集成变声功能，支持用户体验大叔、萝莉等多种不同声线，让沟通成为一种乐此不疲的游戏打造虚拟偶像，释放IP价值一是实现与用户共创合成歌曲，人工智能生成内容（AIGC）白皮书36不断加深粉丝黏性以初音未来和洛天依为代表的“虚拟歌姬”，都是基于', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
 Document(page_content='如欧莱雅、飞利浦、完美日记等品牌的虚拟主播一般会在凌晨0点上线，并进行近9个小时的直播，与真人主播形成了24小时无缝对接的直播服务二是虚拟化的品牌主播更能加速店铺或品牌年轻化进程，拉近与新消费人群的距离，塑造元宇宙时代的店铺形象，未来可通过延展应用到元宇宙中更多元的虚拟场景，实现多圈层传播如彩妆品牌“卡姿兰”推出自己的品牌虚拟形象，并将其引入直播间作为其天猫旗舰店日常的虚拟主播导购', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'})]

In [37]:
query = '虚拟歌手是一种数据要素吗'
docs = retriever.get_relevant_documents(query)

In [38]:
docs

[Document(page_content='和产业更强调数据作为原材料的独特价值，依靠数据驱动可以产生大量人类理性难以直接感知到的信息，这些信息是数据价值释放的一种结果数据要素白皮书（2022年）3来源：公开资料整理图1数据-信息-知识-智慧（DIKW）模型数据与土地、劳动、资本、技术等传统生产要素相比有明显的独特性产学研各界对数据的特性已有广泛讨论，总结各方观点可以发现，数据首先作为独特的技术产物，具有虚拟性、低成本复制性和主体多元性', metadata={'source': '../files/数据要素白皮书-2022.pdf'}),
 Document(page_content='数字人作为现实与虚拟世界的交互媒介，通过其特有的人设、性格、思维、职业等并辅以专属的创作内容，可以打破传统的物理及时空边界，通过VR、AR等技术和新一代智能终端设备，带给用户丰富的沉浸式体验而可以自主生成内容的数字人将是构建人机交融、数实融合的未来世界的最佳载体，同时也是未来人类构建“自生成、自更新”的元宇宙新世界的必经之路', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'}),
 Document(page_content='、鼻型等，可以自动化生成具有个人特色的虚拟形象，同时还可跟踪用户的面部表情生成实时动画，让普通人也能有机会创造属于自己的卡通形象在可预见的未来，作为用户在虚拟世界中个人身份和交互载体的“数字化身”，将进一步与人们的生产生活相融合，并将带动虚拟商品经济的发展（五）AIGC+其他：推进数实融合，加快产业升级除以上行业之外，教育、金融、医疗、工业等各行各业的AIGC应用也都在快速发展教育领域，', metadata={'source': '../files/人工智能生成内容白皮书-2022.pdf'})]